In [1]:
!git clone https://github.com/Adonpm/ai-interviewer.git
%cd ai-interviewer
!pip install -r requirements.txt

Cloning into 'ai-interviewer'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 2), reused 11 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), done.
Resolving deltas: 100% (2/2), done.
/Users/viveksajithasujan/Downloads/ai-interviewer
DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/huggingface_hub-0.29.2-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [5]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAPI_KEY"

In [17]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
import os, asyncio
from dotenv import load_dotenv
load_dotenv()


load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model_client = OpenAIChatCompletionClient(model="gpt-4o")

teacher = AssistantAgent(
    name="Teacher",
    model_client=model_client,
    description="Asks 3 linear algebra questions and explains answers.",
    system_message="""
You are a friendly 8th-grade math teacher.
Ask exactly 3 different linear algebra questions, one at a time.
After each student answer:
1. Say if it is correct only if the answer given by student is correct else say incorrectf.
2. Give the correct answer.
3. Explain step-by-step how to solve it.
Then ask the next question.
After the third question and explanation, say 'TERMINATE'.
Keep explanations under 80 words.
""",
)

kid = UserProxyAgent(
    name="Kid",
    description="The student answering the teacher's questions.",
    input_func=lambda prompt: input(prompt)
)

termination_condition = TextMentionTermination(text="TERMINATE")

team = RoundRobinGroupChat(
    participants=[teacher, kid],
    termination_condition=termination_condition,
    max_turns=20,
)

async def main():
    await Console(team.run_stream(task="Teacher: Please start with the first question."))

import nest_asyncio
nest_asyncio.apply()
await main()

---------- TextMessage (user) ----------
Teacher: Please start with the first question.
---------- TextMessage (Teacher) ----------
Certainly! Here's the first question:

1. Solve for x: 3x + 4 = 19.


Enter your response:  5


---------- TextMessage (Kid) ----------
5
---------- TextMessage (Teacher) ----------
Correct!

To solve 3x + 4 = 19:
1. Subtract 4 from both sides: 3x = 15.
2. Divide both sides by 3: x = 5.

Now, onto the second question:

2. Find the value of y if 2y - 7 = 3.


Enter your response:  4


---------- TextMessage (Kid) ----------
4
---------- TextMessage (Teacher) ----------
Correct!

To solve 2y - 7 = 3:
1. Add 7 to both sides: 2y = 10.
2. Divide both sides by 2: y = 5.

Here's the third question:

3. Solve for z: 5z + 1 = 26.


Enter your response:  3 or 4


---------- TextMessage (Kid) ----------
3 or 4
---------- TextMessage (Teacher) ----------
Incorrect. The correct answer is z = 5.

To solve 5z + 1 = 26:
1. Subtract 1 from both sides: 5z = 25.
2. Divide both sides by 5: z = 5.

TERMINATE
